In [1]:
%reset -f

In [2]:
import pandas as pd
import numpy as np
import os
import yaml
from easydict import EasyDict

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import normalized_mutual_info_score as NMI

from z_common import config_to_execute
from clustering import run_clustering


/home/syamagami/anaconda3/envs/generic/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
N_CUDA = 0
PARENT = 'Office31'
DATA_DOMAIN = 'amazon_dslr_webcam'
log_dir_opt = f''

config, CONFIG_FILE, log_dir = config_to_execute(DATA_DOMAIN, PARENT, N_CUDA, log_dir_opt)

==============  amazon_dslr_webcam  ==============


In [4]:
config.config_path = CONFIG_FILE
config.log_dir = log_dir
config.lap = 1
# feats, edls_dataset = run_clustering(config)

In [8]:
data_domain = '_'.join(config.dataset.target_dsets)
data_domain

'amazon_dslr_webcam'

In [5]:
# 正解率
dfs = []
dfs.append(pd.read_csv(os.path.join(log_dir, 'real_domain.csv'), names=['label'], dtype=int))
dfs.append(pd.read_csv(os.path.join(log_dir, 'cluster_pca_gmm.csv'), names=['label'], dtype=int))
dfs.append(pd.read_csv(os.path.join(log_dir, 'real_class.csv'), names=['label'], dtype=int))
dfs.append(pd.read_csv(os.path.join(log_dir, 'cluster_pca_gmm_class.csv'), names=['label'], dtype=int))

df = pd.concat(dfs, axis=1)
df.columns = ['rdomain', 'edomain', 'rclass', 'eclass']
df

FileNotFoundError: [Errno 2] No such file or directory: 'record/Office31/CUDA0/adw_resnet/real_domain.csv'

In [ ]:
print(  NMI(df.rdomain, df.edomain)  )
print(  100.0 * len(df[df.rdomain == df.edomain]) / len(df)  )
print(  NMI(df.rclass, df.eclass)  )
print(  100.0 * len(df[df.rclass == df.eclass]) / len(df)  )

In [ ]:
from sklearn.metrics import confusion_matrix

cmx = pd.DataFrame(confusion_matrix(df.rdomain, df.edomain), index = ['r_amazon', 'r_dslr', 'r_webcam'],  columns = ['e_amazon', 'e_dslr', 'e_webcam'])
cmx['total'] = cmx.sum(axis=1)
display(cmx)
